# Série 2 - Exercice 1

On va faire quelques requêtes SSL vers des sites pour voir comment ils répondent.

## 1. Connaissance

La façon la plus simple de faire ces connexions est en fait d'utiliser le navigateur. 
Autant Mozilla Firefox, Chrome, Safari, Edge, vous permettent de montrer les certificats
utilisés pour sécuriser le site visité.

Donc pour cette première partie, visitez les sites suivants avec votre navigateur, et trouvez:
- le nombre de certificats dans la liste
- le certificat de base

Les sites sont:
- https://google.ch
- https://google.com
- https://moodle.unifr.ch/
- https://web.fledg.re

Est-ce que vous trouvez le site qui utilise Let's Encrypt?

## 2. Compréhension

Utiliser le navigateur est simple, mais en utilisant Python on voit un peu plus comment ça marche.
Dans l'exemple en bas, il y a quelques méthods qui ont été créées, tout d'abord ceux-ci:

- ´get_connection_chain´ se connecte au serveur indiqué et retourne la connection et la chaîne de certificats - ATTENTION: il faut seulement donner le nom de domaine, pas l'URL complète!
- ´dump_cert´ imprime un certificat
- ´dump_chain´ imprime toute la chaîne

Qu'est-ce que vous observez en lançant le code? Pourquoi c'est différent?

**Si on regarde la chaîne de certificats pour `google.ch` ou `google.com`, on voit que le certificat racine
est différent que celui affiché par le navigateur.**

**Je soupçonne que les serveurs google retournent une chaîne différente selon le client qui fait la requête.
Si c'est un client Chrome ou Firefox, il peut retourner une chaîne avec un certificat signé par lui-même.
Sinon il retourne un certificat signé par une autorité reconnue.**

**Vous pouvez suivre la résolution de ce mystère ici: 
https://stackoverflow.com/questions/70157494/is-google-returning-different-certificates-for-different-clientts**

Grâce au Python, nous avons aussi de l'information supplémentaire sur la connexion. Essayez les deux adresses suivantes:
- https://xkcd.com
- https://sssscomic.com


In [ ]:
# Série 2 - Exercice 1 - Partie 2

from OpenSSL import SSL, crypto
import socket, certifi

def dump_cert(cert):
    for component in cert.get_subject().get_components():
        print("Subject %s: %s" % (component))
             
    print("notBefore:", cert.get_notBefore())
    print("notAfter:", cert.get_notAfter())
    print("version:" + str(cert.get_version()))
    print("sigAlg:", cert.get_signature_algorithm())
    print("digest:", cert.digest('sha256'))
    print("issuer:", cert.get_issuer())
    print()
    
def get_connection_chain(host, port = 443):
    dst = (str.encode(host), port)
    ctx = SSL.Context(SSL.TLSv1_2_METHOD)
    s = socket.create_connection(dst)
    s = SSL.Connection(ctx, s)
    s.set_connect_state()
    s.set_tlsext_host_name(dst[0])

    s.sendall(b'HEAD / HTTP/1.2\n\n')
    s.recv(16)
    return (s, s.get_peer_cert_chain())

def dump_chain(chain):
    for pos, cert in enumerate(chain):
        print("Certificate #" + str(pos))
        dump_cert(cert)
        
def dump_connection(conn):
    print("Cipher name:", conn.get_cipher_name())
    print("Cipher size:", conn.get_cipher_bits())
    print("Cipher version:", conn.get_cipher_version())
    print("Cipher list:", conn.get_cipher_list())

conn, chain = get_connection_chain("google.ch")
dump_chain(chain)
dump_connection(conn)

## 3. Application

Pour la troisième partie, on va enfin aller chercher le certificat de base. Comme on a vu dans la partie 2, le certificat de base n'est pas envoyé par le site. Ca donne du sens, parce qu'on ne fait à priori pas confiance au site. Donc on ne va pas accepter un certificat qui vient de là.

Il faut donc avoir une liste de certificats à qui on fait confiance. Cette liste réside sur votre ordinateur, et peut être récupérée avec ´certifi.where()´.

Vous pouvez maintenant utiliser la méthode ´get_root_cert´ sur un des certificats précédents pour vous retourner la chaîne de certificats de base.

Question:
- pourquoi on donne seulement le dernier élément de la liste (`chain[-1]`)?

** Parce que les autres éléments sont signés par leur élément précédent, donc on ne pourra pas les
vérifier avec les certificats enregistrés localement.**

- pourquoi on reçoit deux certificats à la fin?

** On reçoit le certificat donné en argument, plus le certificat racine. Comme ça on peut vérifier que c'est bien
le certificat racine qui a signé le certificat en question.**

In [ ]:
# Série 2 - Exercice 1 - Partie 3

def get_root_chain(chain):
    store = crypto.X509Store()
    store.load_locations(certifi.where())
    store_ctx = crypto.X509StoreContext(store, chain[-1])
    return store_ctx.get_verified_chain()

conn, chain = get_connection_chain("google.ch")
dump_chain(get_root_chain(chain))
